# Create Image with Model 

Download some realife images.

Like this [6gb coco data](http://images.cocodataset.org/zips/val2014.zip)

Or this [19gb coco data](http://images.cocodataset.org/zips/unlabeled2017.zip) will work even better

In [20]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader

### A spacialized dataset for the task

In [86]:
from glob import glob
from PIL import Image
class super_res(Dataset):
    def __init__(self, root_dir):
        """
        A dataset to spit out low and high version of the same picture
        root_dir: directory of the images
        
        """
        self.root_dir = root_dir if root_dir[-1]=="/" else root_dir+"/"
        
        self.resize1 = transforms.Resize((128,128)) # function: resize to smaller
        self.resize2 = transforms.Resize((256,256)) # function: resize to bigger
        
        self.data_aug = transforms.Compose([
            transforms.RandomHorizontalFlip(), # function flip
            transforms.RandomAffine(5), # function rotate
                              ])
        self.toTensor = transforms.ToTensor()
        self.imgs = glob(self.root_dir + "*.jpg")
        
    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self,idx):
        img_name = self.imgs[idx]
        img = Image.open(img_name,mode = "r").convert('RGB')
        image_l = self.toTensor(self.resize1(self.data_aug(img))) # Low resolution version
        image_h = self.toTensor(self.resize2(self.data_aug(img))) # High resolution version

        return image_l, image_h

In [87]:
DATA = "/data/unlabeled2017/"

In [88]:
ds.imgs[:3]

['/data/unlabeled2017/000000437307.jpg',
 '/data/unlabeled2017/000000227544.jpg',
 '/data/unlabeled2017/000000388209.jpg']

In [89]:
ds.toTensor(ds.resize1(ds.data_aug(Image.open(ds.imgs[1]))))

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.2353,  0.2667,  0.1137],
         [ 0.0863,  0.0941,  0.1020,  ...,  0.4275,  0.4706,  0.2118],
         [ 0.1647,  0.1804,  0.2078,  ...,  0.5412,  0.6667,  0.2863],
         ...,
         [ 0.3725,  0.7765,  0.8196,  ...,  0.8314,  0.8275,  0.7961],
         [ 0.3137,  0.8118,  0.8157,  ...,  0.4118,  0.4118,  0.4157],
         [ 0.3451,  0.7647,  0.8039,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.2471,  0.2706,  0.1137],
         [ 0.0627,  0.0706,  0.0863,  ...,  0.4275,  0.4118,  0.1804],
         [ 0.1294,  0.1451,  0.1686,  ...,  0.4824,  0.5804,  0.2314],
         ...,
         [ 0.2980,  0.6118,  0.6706,  ...,  0.6941,  0.6902,  0.6588],
         [ 0.2588,  0.6627,  0.6667,  ...,  0.3451,  0.3451,  0.3451],
         [ 0.2745,  0.6039,  0.6471,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.2510,  0.2667,  0.1137],
         [ 0.0588,  0.0706,  0.0784,  ...,  0

In [90]:
ds = super_res(DATA)
dl = DataLoader(ds,batch_size=1,shuffle = True)
generator = iter(dl)
next(generator)
# len(dl)

[tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0039,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0039,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0039,  ...,  0.0000,  0.0000,  0.0000],
           ...,
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0039,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
          [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 

In [11]:
from torch import nn

In [14]:
def conv_layer(in_,out_,ks, stride,activation = True):
    layers = [
        nn.Conv2d(in_,out_,kernel_size = ks, stride = stride, padding = ks//2, bias = False),
        nn.BatchNorm2d(out_),
    ]
    if activation: layers.append(nn.LeakyReLU())
    return nn.Sequential(*layers)

In [15]:
conv_layer(3,4,3,1)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
)

In [17]:
class res_block(nn.Module):
    def __init__(self,nb_filter):
        """
        simple resnet block
        """
        super(res_block,self).__init__()
        self.nb_filter = nb_filter
        self.clayer1 = conv_layer(self.nb_filter,self.nb_filter,3,(1,1))
        self.clayer2 = conv_layer(self.nb_filter,self.nb_filter,3,(1,1),activation = False)
    
    def forward(self,x):
        x2 = self.clayer2(self.clayer1(x))
        return x2+x1